In [13]:
# %env PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
# %env  PATH_FILE_PRIVATE_KEY_ZEROMQ_SERVER=/home/adam/Engineering/Docker/smart_broker/private_keys/server.key_secret
# %env PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys
# %env PATH_DIR_DATASET=/home/adam/datascience/data

env: PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
env: PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys


In [1]:
class Session():
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port

session = Session("127.0.0.1", "8000")

# 1. Data Model Creation

In [2]:
from smart_broker_api import data_model_api
data_model_id = data_model_api.create_date_frame(session, "data_frame_0")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_mean", "Observation:Body Mass Index")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_first", "Observation:Body Mass Index")
data_model_api.data_frame_add_series(session, data_model_id, "bmi_last", "Observation:Body Mass Index")
data_model_tabular_id = data_model_api.create_tabular_data(session)
data_model_tabular_id = data_model_api.tabular_add_dataframe(session, data_model_id, data_model_tabular_id)

# 2. Parse Data from Data Model Specification

In [3]:
from smart_broker_api import data_api

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"

longitudinal_id = data_api.read_longitudinal_fhirv1(session)
tabular_dataset_id = data_api.parse_dataset_tabular_from_longitudinal(session, longitudinal_id, dataset_federation_id, dataset_federation_name, data_model_tabular_id)
data_frame_id = data_api.data_frame_tabular_select_data_frame(session, tabular_dataset_id, "data_frame_0")

# 3. Grab series to work with in Statistics Lib

In [4]:
series_1_id = data_api.data_frame_select_series(session, data_frame_id, "bmi_mean")
series_2_id = data_api.data_frame_select_series(session, data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

83527a08-eb6d-4d25-9b66-cda5d5ef435f , 159246a8-011a-4894-847c-99dc6a580d10


# 4. Run Statistics Library

In [6]:
from smart_broker_api import statistics_api

type_distribution="normalunit" 
type_ranking="unsafe"
alternative = "two-sided"
print(statistics_api.count(session,  series_1_id))
print(statistics_api.mean(session,  series_1_id))
print(statistics_api.chisquare(session,  series_1_id, series_2_id))
# print(kolmogorovSmirnovTest(session,  series_1_id, type_distribution, type_ranking)) # RANK DATAFRAME NOT FOUND
print(statistics_api.kurtosis(session,  series_1_id))
# print(levene_test(session,  series_1_id, series_2_id)) # NDARRAY SERIALISATION
# mann_whitney_u_test(session,  series_1_id, series_2_id, alternative, type_ranking)  # RANK DATAFRAME NOT FOUND
print(statistics_api.min_max(session,  series_1_id))
# print(paired_t_test(session,  series_1_id, series_2_id, alternative)) # ValueError: Out of range float values are not JSON compliant
# print(pearson(session,  series_1_id, series_2_id, alternative)) # ValueError: math domain error
print(statistics_api.skewness(session,  series_1_id))
# print(spearman(session,  series_1_id, series_2_id, alternative, type_ranking))  # RANK DATAFRAME NOT FOUND
print(statistics_api.student_t_test(session,  series_1_id, series_2_id, alternative))
print(statistics_api.variance(session,  series_1_id))
print(statistics_api.welch_t_test(session,  series_1_id, series_2_id, alternative))
# # # print(wilcoxon_signed_rank_test(session,  series_1_id, series_2_id, alternative, type_ranking)) # RANK DataFrame not loaded: 30459bd2-2880-4829-8d05-1e6bb9c1a71f







20
22.76433814224586
[380.0, 0.23583148842099294]
-1.225866948308427
(11.76951154694196, 30.172100539128525)
-0.08041628194553505
(0.0, 0.25)
28.617281263722433
(0.0, 0.25)
